### Ноутбук необходим для загрузки новых данных офд и проверки новых адресов с имеющейся для добавления новых

In [1]:
import pandas as pd
import numpy as np

In [109]:
df_ofd = pd.read_csv('/Users/rollan/Desktop/ofd/data/ktelecom_data/04.05.2022-10.05.2022.zip', dtype={
                    'KkmsRetailPlaceOrgCatalog_orgInn': 'str'},
                    usecols = ['KkmsRetailPlaceOrgCatalog_orgInn',
                              'KkmsRetailPlaceOrgCatalog_retailPlaceAddressRegionCode',
                              'KkmsRetailPlaceOrgCatalog_retailPlaceAddressArea', 
                              'KkmsRetailPlaceOrgCatalog_retailPlaceAddressCity',
                              'KkmsRetailPlaceOrgCatalog_retailPlaceAddressVillage', 
                              'KkmsRetailPlaceOrgCatalog_retailPlaceAddressStreet',
                              'KkmsRetailPlaceOrgCatalog_retailPlaceAddressHouse'])

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (9,11,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [110]:
kz_geocode_last = pd.read_csv('../data/processed_data/kaz_geocash.csv',index_col=0)

In [111]:
dict_region = pd.read_csv('../data/interim_data/region_dict.csv')

In [112]:
def process_data(ofd_data):
    """
    Returns the unique addresses of the last week ofd raw data

    Parameters:
        ofd_data: ofd raw dataset

    Returns:
        ofd_data_clean: unique addresses of ofd dataset
    """
    ofd_data.drop_duplicates(subset = ['KkmsRetailPlaceOrgCatalog_retailPlaceAddressArea',
                        'KkmsRetailPlaceOrgCatalog_retailPlaceAddressCity',
                        'KkmsRetailPlaceOrgCatalog_retailPlaceAddressVillage',
                        'KkmsRetailPlaceOrgCatalog_retailPlaceAddressStreet',
                        'KkmsRetailPlaceOrgCatalog_retailPlaceAddressHouse'], inplace=True)
    ofd_data['full_add'] = ofd_data[ofd_data.columns[2:]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
    ofd_data.drop_duplicates(subset = ['full_add'], inplace=True)
    ofd_data_clean = ofd_data.merge(dict_region[['code','region']],how='left',
                        left_on='KkmsRetailPlaceOrgCatalog_retailPlaceAddressRegionCode',
                        right_on='code')
    ofd_data_clean.rename(columns={'KkmsRetailPlaceOrgCatalog_orgInn':'bin'}, inplace=True)
    ofd_data_clean = ofd_data_clean[['bin','code','region','full_add']]
    return ofd_data_clean

In [114]:
%%time
df_ofd_process = process_data(df_ofd)

CPU times: user 52.9 s, sys: 23.1 s, total: 1min 15s
Wall time: 1min 25s


#### Загрузка базы кодов с геокоординатами и сравнение отличий

In [115]:
def address_check(new_ofd,current_database):
    """
    Return addresses which are not in geocoding dataset

    Parameters:
        new_ofd: unique addresses of ofd dataset
        current_database: full geocoding dataset

    Returns:
        new_ofd_last: not macthed addresses 
    """    
    set_new_ofd = set(new_ofd['full_add'].values)
    set_current_database = set(current_database['full_add'].values)
    
    diff_ofd_base = list(set_new_ofd.difference(set_current_database))
    new_ofd_last = new_ofd[new_ofd['full_add'].isin(diff_ofd_base)]
    
    new_ofd_last.reset_index(drop=True,inplace=True)
    return new_ofd_last

In [116]:
df_ofd_unique_cut_region = address_check(df_ofd_process,kz_geocode_last)

In [118]:
df_ofd_unique_cut_region.shape

(41697, 4)

In [101]:
df_ofd_unique_cut_region.to_csv('data/to_geocode_data/diff_address_17.05.2022.csv')